## 대신증권 API cybos plus 사용하기

### 필요 모듈 import

In [ ]:
import win32com.client
import pandas as pd
import time
import datetime

### cybos plus 연결 체크

In [2]:
 # 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()
else:
    print("PLUS가 정상적 연결됨")


PLUS가 정상적 연결됨


#### KOSPI 50에 해당하는 종목코드 찾기

In [3]:
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
group_list = instCpCodeMgr.GetIndustryList()
category_info = pd.DataFrame(columns=["category_code", "category_name", "company_nums", "company_list"])
for group in group_list:
    company_list = instCpCodeMgr.GetGroupCodeList(group)
    category_info = category_info.append({"category_code": group, 
                                          "category_name": instCpCodeMgr.GetGroupName(group), 
                                          "company_nums": len(company_list),
                                          "company_list": list(company_list)
                                         }, ignore_index=True)
category_info = category_info[category_info.company_nums != 0]
kospi50_codes = list(category_info[category_info.category_name == "KOSPI50지수"].company_list)[0]
kospi50_codes
print(len(kospi50_codes), '\n', kospi50_codes)

50 
 ['A105560', 'A030200', 'A033780', 'A003550', 'A034220', 'A051900', 'A032640', 'A066570', 'A051910', 'A035420', 'A005490', 'A010950', 'A034730', 'A302440', 'A326030', 'A361610', 'A096770', 'A017670', 'A000660', 'A035250', 'A010130', 'A000270', 'A024110', 'A251270', 'A011170', 'A006400', 'A028260', 'A207940', 'A032830', 'A018260', 'A009150', 'A005930', 'A000810', 'A068270', 'A055550', 'A090430', 'A036570', 'A316140', 'A035720', 'A021240', 'A003670', 'A086790', 'A352820', 'A015760', 'A009540', 'A161390', 'A018880', 'A086280', 'A012330', 'A005380']


In [4]:
stockitems = pd.read_csv("./data/stockitems.csv", index_col="code")
kospi50_info = stockitems.loc[list(kospi50_codes)]
kospi50_info = kospi50_info.reset_index()
kospi50_info

,code,name,section,sectionKind,group_category
0,A105560,KB금융,KOSPI,1,21
1,A030200,KT,KOSPI,1,20
2,A033780,KT&G,KOSPI,1,2
3,A003550,LG,KOSPI,1,21
4,A034220,LG디스플레이,KOSPI,1,13
5,A051900,LG생활건강,KOSPI,1,8
6,A032640,LG유플러스,KOSPI,1,20
7,A066570,LG전자,KOSPI,1,13
8,A051910,LG화학,KOSPI,1,8
9,A035420,NAVER,KOSPI,1,26


### KOSPI 50 모든 데이터 가져오는 함수
- 시가 / 고가 / 저가 / 종가

In [5]:
def get_kospi50_stock_data(stockitems, start, end):
    instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    
    # 날짜, 종목명, 시가, 고가, 저가, 종가
    column_dailychart = ['code', 'name', 'date', 'open', 'high', 'low', 'close',]
    
    stockitems_dailychart = pd.DataFrame()
    
    s_year, e_year = start[:4], end[:4]
    s_day, e_day = start[4:], end[4:]
    
    instStockChart.SetInputValue(1, ord('1'))
    instStockChart.SetInputValue(5, (0, 2, 3, 4, 5))
    instStockChart.SetInputValue(6, ord('D'))
    instStockChart.SetInputValue(9, ord('1'))

    for idx, stockitem in stockitems.iterrows():
        dailychart = pd.DataFrame()
        row = list(range(len(column_dailychart)))
        rows = list()
        
        instStockChart.SetInputValue(0, stockitem['code'])
        print(stockitem['name'], "주식 데이터를 가져옵니다.")
        
        y = int(s_year)
        while y <= int(e_year):
            # 시작일, 종료일 계산
            if s_year == y:
                s_y, s_d = s_year, s_day
            else:
                s_y, s_d = str(y), '0101'
                
            if e_year == y:
                e_y, e_d = e_year, e_day
            else:
                e_y, e_d = str(y), '1231'
                
            print(s_y + s_d + '~' + e_y + e_d + '의 데이터를 가져옵니다.')
            instStockChart.SetInputValue(3, s_y + s_d)
            instStockChart.SetInputValue(2, e_y + e_d)
            
            # 남은요청 있는지 확인
            remain_request_count = nCpCybos.GetLimitRemainCount(1)
            print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)

            if remain_request_count == 0:
                print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')
                while True:
                    time.sleep(2)
                    remain_request_count = nCpCybos.GetLimitRemainCount(1)

                    if remain_request_count > 0:
                        print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                        break
                    print('대기 중...')
                    
            # BlockRequest
            instStockChart.BlockRequest()

            # GetHeaderValue
            numData = instStockChart.GetHeaderValue(3)
            numField = instStockChart.GetHeaderValue(1)
            # GetDataValue
            print(numData)
            for i in range(numData):
                row[0] = stockitem['code']
                row[1] = stockitem['name']
                row[2] = instStockChart.GetDataValue(0, i)# 날짜
                row[3] = instStockChart.GetDataValue(1, i) # 시가
                row[4] = instStockChart.GetDataValue(2, i) # 고가
                row[5] = instStockChart.GetDataValue(3, i) # 저가
                row[6] = instStockChart.GetDataValue(4, i) # 종가
                rows.append(list(row))
            df =  pd.DataFrame(data = rows, columns= column_dailychart)
            dailychart = pd.concat([dailychart, df])
            y += 1    
        dailychart = dailychart.sort_values("date")
        dailychart = dailychart.drop_duplicates(['date'])
        dailychart = dailychart.reset_index(drop=True)
        stockitems_dailychart = pd.concat([stockitems_dailychart, dailychart])
    return stockitems_dailychart

In [6]:
kospi50_daily_stock_chart = get_kospi50_stock_data(kospi50_info, "19800101", "20211231")

KB금융 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  60
0
19810101~19811231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  59
0
19820101~19821231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  58
0
19830101~19831231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  57
0
19840101~19841231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  56
0
19850101~19851231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  55
0
19860101~19861231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  54
0
19870101~19871231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  53
0
19880101~19881231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  52
0
19890101~19891231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  51
0
19900101~19901231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  50
0
19910101~19911231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  49
0
19920101~19921231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  48
0
19930101~19931231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  47
0
19940101~19941231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  46
0
19950101~19951231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 :  45
0
19960101~19961231의 데이터를 가져옵니다.
A105560 KB금융 남은 요청 : 

A003550 LG 남은 요청 :  42
294
19930101~19931231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  41
296
19940101~19941231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  40
296
19950101~19951231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  39
293
19960101~19961231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  38
293
19970101~19971231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  37
292
19980101~19981231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  36
292
19990101~19991231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  35
249
20000101~20001231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  34
241
20010101~20011231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  33
246
20020101~20021231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  32
244
20030101~20031231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  31
247
20040101~20041231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  30
249
20050101~20051231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  29
249
20060101~20061231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  28
247
20070101~20071231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  27
246
20080101~20081231의 데이터를 가져옵니다.
A003550 LG 남은 요청 :  26
248
20090101~20091231의 데이터를 가져옵니다.
A003550 LG 남은 

20030101~20031231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  25
247
20040101~20041231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  24
249
20050101~20051231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  23
249
20060101~20061231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  22
247
20070101~20071231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  21
246
20080101~20081231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  20
248
20090101~20091231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  19
253
20100101~20101231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  18
251
20110101~20111231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  17
248
20120101~20121231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  16
248
20130101~20131231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  15
247
20140101~20141231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  14
245
20150101~20151231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  13
248
20160101~20161231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  12
246
20170101~20171231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  11
243
20180101~20181231의 데이터를 가져옵니다.
A032640 LG유플러스 남은 요청 :  10
244
20190101

20110101~20111231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  11
248
20120101~20121231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  10
248
20130101~20131231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  9
247
20140101~20141231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  8
245
20150101~20151231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  7
248
20160101~20161231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  6
246
20170101~20171231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  5
243
20180101~20181231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  4
244
20190101~20191231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  3
246
20200101~20201231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  2
248
20210101~20211231의 데이터를 가져옵니다.
A035420 NAVER 남은 요청 :  1
171
POSCO 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A005490 POSCO 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
0
19810101~19811231의 데이터를 가져옵니다.
A005490 POSCO 남은 요청 :  59
0
19820101~19821231의 데이터를 가져옵니다.
A005490 POSCO 남은 요청 :  58
0
19830101~19831231의 데이터를 가져옵니다.
A005490 P

20200101~20201231의 데이터를 가져옵니다.
A034730 SK 남은 요청 :  56
248
20210101~20211231의 데이터를 가져옵니다.
A034730 SK 남은 요청 :  55
171
SK바이오사이언스 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  54
0
19810101~19811231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  53
0
19820101~19821231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  52
0
19830101~19831231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  51
0
19840101~19841231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  50
0
19850101~19851231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  49
0
19860101~19861231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  48
0
19870101~19871231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  47
0
19880101~19881231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  46
0
19890101~19891231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  45
0
19900101~19901231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  44
0
19910101~19911231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  43
0
19920101~19921231의 데이터를 가져옵니다.
A302440 SK바이오사이언스 남은 요청 :  42
0
19930101~19931231의 데이터를 가져옵니다.
A302440 S

...

A090430 아모레퍼시픽 남은 요청 :  22
0
19890101~19891231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  21
0
19900101~19901231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  20
0
19910101~19911231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  19
0
19920101~19921231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  18
0
19930101~19931231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  17
0
19940101~19941231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  16
0
19950101~19951231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  15
0
19960101~19961231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  14
0
19970101~19971231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  13
0
19980101~19981231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  12
0
19990101~19991231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  11
0
20000101~20001231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  10
0
20010101~20011231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  9
0
20020101~20021231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  8
0
20030101~20031231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  7
0
20040101~20041231의 데이터를 가져옵니다.
A090430 아모레퍼시픽 남은 요청 :  6
0
20050101~200512

0
19990101~19991231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  5
34
20000101~20001231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  4
241
20010101~20011231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  3
246
20020101~20021231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  2
244
20030101~20031231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  1
247
20040101~20041231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
249
20050101~20051231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  59
249
20060101~20061231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  58
247
20070101~20071231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  57
246
20080101~20081231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  56
248
20090101~20091231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  55
253
20100101~20101231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  54
251
20110101~20111231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  53
248
20120101~20121231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  52
248
20130101~20131231의 데이터를 가져옵니다.
A035720 카카오 남은 요청 :  51
247
20140101~20141231의 

248
20090101~20091231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  49
253
20100101~20101231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  48
251
20110101~20111231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  47
248
20120101~20121231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  46
248
20130101~20131231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  45
247
20140101~20141231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  44
245
20150101~20151231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  43
248
20160101~20161231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  42
246
20170101~20171231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  41
243
20180101~20181231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  40
244
20190101~20191231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  39
246
20200101~20201231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  38
248
20210101~20211231의 데이터를 가져옵니다.
A086790 하나금융지주 남은 요청 :  37
171
하이브 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A352820 하이브 남은 요청 :  36
0
19810101~19811231의 데이터를 가져옵니다.
A352820 하이브 남은 요청 :  35
0
19820101~19821231의 데이터를 가져옵니다.
A352820 하이브 남은 요청 :  34
0


A009540 한국조선해양 남은 요청 :  33
246
20200101~20201231의 데이터를 가져옵니다.
A009540 한국조선해양 남은 요청 :  32
248
20210101~20211231의 데이터를 가져옵니다.
A009540 한국조선해양 남은 요청 :  31
171
한국타이어앤테크놀로지 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  30
0
19810101~19811231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  29
0
19820101~19821231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  28
0
19830101~19831231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  27
0
19840101~19841231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  26
0
19850101~19851231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  25
0
19860101~19861231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  24
0
19870101~19871231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  23
0
19880101~19881231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  22
0
19890101~19891231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  21
0
19900101~19901231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  20
0
19910101~19911231의 데이터를 가져옵니다.
A161390 한국타이어앤테크놀로지 남은 요청 :  19
0
19920101~19921231의 데이터를 가져옵니다.
A161390

A012330 현대모비스 남은 요청 :  22
0
19830101~19831231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  21
0
19840101~19841231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  20
0
19850101~19851231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  19
0
19860101~19861231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  18
0
19870101~19871231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  17
0
19880101~19881231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  16
0
19890101~19891231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  15
90
19900101~19901231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  14
291
19910101~19911231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  13
292
19920101~19921231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  12
294
19930101~19931231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  11
296
19940101~19941231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  10
296
19950101~19951231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  9
293
19960101~19961231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  8
293
19970101~19971231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  7
292
19980101~19981231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  6
292
19990101~1999

In [7]:
kospi50_daily_stock_chart

,code,name,date,open,high,low,close
0,A105560,KB금융,20081010,47063,47063,40026,45939
1,A105560,KB금융,20081013,47063,50338,45499,49556
2,A105560,KB금융,20081014,52781,54736,50631,51902
3,A105560,KB금융,20081015,49067,50142,48578,49849
4,A105560,KB금융,20081016,44620,44913,42420,42420
...,...,...,...,...,...,...,...
11163,A005380,현대차,20210831,210000,212500,208000,212500
11164,A005380,현대차,20210901,214000,218500,213500,215000
11165,A005380,현대차,20210902,214000,215000,210000,211500
11166,A005380,현대차,20210903,213000,215500,212000,214500


#### KOSPI 50 지수 index 데이터 가져오기

In [8]:
kospi50index = pd.DataFrame(data=[["U182", "kospi50_index", "KOSPI"]], columns=["code", "name", "section"])
kospi50index

,code,name,section
0,U182,kospi50_index,KOSPI


In [9]:
kospi50index_data = get_kospi50_stock_data(kospi50index, "19800101", "20211231")
kospi50index_data

kospi50_index 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
0
19810101~19811231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  59
0
19820101~19821231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  58
0
19830101~19831231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  57
0
19840101~19841231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  56
0
19850101~19851231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  55
0
19860101~19861231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  54
0
19870101~19871231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  53
0
19880101~19881231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  52
0
19890101~19891231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  51
0
19900101~19901231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  50
0
19910101~19911231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  49
0
19920101~19921231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  48
0
19930101~19931231의 데이터를 가져옵니다.
U182 kospi50_ind

,code,name,date,open,high,low,close
0,U182,kospi50_index,20000302,804.739990,833.580017,786.229980,826.950012
1,U182,kospi50_index,20000303,843.460022,845.299988,816.580017,832.000000
2,U182,kospi50_index,20000306,844.330017,849.239990,831.119995,836.200012
3,U182,kospi50_index,20000307,831.460022,833.369995,816.719971,833.349976
4,U182,kospi50_index,20000308,823.349976,847.530029,810.869995,844.239990
...,...,...,...,...,...,...,...
5309,U182,kospi50_index,20210831,2858.959961,2920.040039,2845.699951,2919.100098
5310,U182,kospi50_index,20210901,2917.189941,2927.479980,2898.419922,2917.949951
5311,U182,kospi50_index,20210902,2912.090088,2917.959961,2880.020020,2888.439941
5312,U182,kospi50_index,20210903,2899.250000,2916.439941,2891.510010,2915.719971


#### KOSPI index와 개별 주식 데이터 합치고 저장하기

In [10]:
kospi50_index_stocks = pd.concat([kospi50index_data, kospi50_daily_stock_chart])
kospi50_index_stocks

,code,name,date,open,high,low,close
0,U182,kospi50_index,20000302,804.74,833.58,786.23,826.95
1,U182,kospi50_index,20000303,843.46,845.3,816.58,832
2,U182,kospi50_index,20000306,844.33,849.24,831.12,836.2
3,U182,kospi50_index,20000307,831.46,833.37,816.72,833.35
4,U182,kospi50_index,20000308,823.35,847.53,810.87,844.24
...,...,...,...,...,...,...,...
11163,A005380,현대차,20210831,210000,212500,208000,212500
11164,A005380,현대차,20210901,214000,218500,213500,215000
11165,A005380,현대차,20210902,214000,215000,210000,211500
11166,A005380,현대차,20210903,213000,215500,212000,214500


In [11]:
kospi50_index_stocks.to_csv("./data/kospi50-1980-2021.csv", index=False)